In [1]:
import pandas as pd
import numpy as np


from sqlalchemy import create_engine


ck_properties = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "5NsWWQeJ",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_properties_dev = {  # mysql 开发环境配置
    "host": "10.179.98.13",
    "port": "3306",
    "url": "jdbc:mysql://10.179.98.13:3306/ck",
    "user": "root",
    "password": "DMP_test135#",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

ck_engine = create_engine(
    f"clickhouse://{ck_properties['user']}:{ck_properties['password']}@{ck_properties['host']}:{ck_properties['port']}/{ck_properties['database']}"
)

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)

mysql_engine_dev = create_engine(
    f"mysql+pymysql://{mysql_properties_dev['user']}:{mysql_properties_dev['password']}" +
    f"@{mysql_properties_dev['host']}:{mysql_properties_dev['port']}/{mysql_properties_dev['database']}?charset=utf8"
)


In [2]:
import os
os.path.abspath('.')

'/data1/pythonProjects/pyetl/core/notebooks'

In [3]:
# 读取NOTCCPPARAINFO
para_df = pd.read_sql(
    sql=f"select * from ods.eol_notccpparainfo",
    con=mysql_engine_prod
)
print(para_df.columns)
para_df = para_df.loc[:, ["PID", "Description", "ECUName", "name", "unitname"]]
para_df.head()

Index(['ID', 'PID', 'ECUName', 'AcqstEnble', 'GetMode', 'UploadEnable',
       'Uploadmode', 'Uploadtime', 'Channel', 'Style', 'ReqAddr', 'ResAddr',
       'SendDataCnt', 'SendData', 'ReqTime', 'Description', 'name', 'unitname',
       'englishDescription', 'startbyte', 'startbit', 'bitnum', 'bitseq', 'B',
       'C', 'dtcmode', 'dtcnummark', 'headernum', 'dtcbyte', 'dtcstate',
       'dtcmaxnum', 'curdtcmark', 'Bz1', 'Bz2', 'pkgname', 'maxval', 'minval',
       'active', 'group_code', 'signal_type'],
      dtype='object')


,PID,Description,ECUName,name,unitname
0,C81_U010100,发动机转速,C81,Epm_nEng,rpm
1,C81_U010300,实际轨压,C81,RailP_pLin,hPa
2,C81_U010400,空调压缩机状态(Dec_value),C81,ACCtl_stOut,-
3,C81_U010500,EGR冷却器下游温度传感器电压,C81,Air_uRawTEGRClrDs,mV
4,C81_U010600,EGR出口温度,C81,Air_tSensTEGRClrDs,℃


In [4]:
# 读取已经整理的功能 PID 对应关系
app_pid_df = pd.read_excel(
    "/home/qjy20472/pythonProjects/pyetl/data/研发项目管理.xlsx",
    sheet_name="模型信号清单表"
)
print(app_pid_df.columns)
app_pid_df = app_pid_df.loc[:, ['统一信号名(无单位)', 'Confirm PIDList', '关联']]
app_pid_df.columns = ['target_name', 'pids', 'app']
app_pid_df

Index(['统一信号名', '统一信号名(无单位)', '信号名（以D系列为基准)', 'Confirm PIDList', '原 PID',
       '配置文件DC1W', '配置文件MD1', '配置文件C81', '变更备注', 'DC1W', 'MD1', 'C81', 'CV51',
       'F17', 'SECM112', '国标数据', '跃进信号', '红岩信号', '常州东风信号', '信号引用次数3',
       '车辆基本信息参数', '发动机基本运行参数', '国标要求参数', '标定数据FOTA参数', 'TBOX注册激活参数', '关联',
       '信号引用功能数'],
      dtype='object')


/home/qjy20472/miniconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,target_name,pids,app
0,EIN,/,"大屏监控, 发电机性能监测, 加油行为识别"
1,VIN,/,发电机性能监测
2,标定数据零件号,/,NaN
3,标定数据版本号,/,NaN
4,软件标定识别号,/,NaN
...,...,...,...
140,尿素泵温度,NaN,NaN
141,GOC入口温度,SAE_J004753,路谱报表
142,GOC出口温度,SAE_J004754,路谱报表
143,总气耗,SAE_J520504,NaN


In [5]:
# 对已经整理的信号清单，分割pid，匹配中文名
def get_app_name(app):
    if isinstance(app, float):
        return None
    else:
        return app

app_pid_flatten_list = []
for _, row in app_pid_df.iterrows():
    if row['pids'] == "/":
        # 无对应pid跳过该条
        continue

    if isinstance(row['pids'], float):
        continue

    for pid in row['pids'].split('、'):
        app_name = get_app_name(row['app'])
        app_pid_flatten_list.append([row['target_name'], pid, app_name])
app_pid_flatten_df = pd.DataFrame(app_pid_flatten_list, columns=['target_desc', 'pid', 'app'])
app_pid_flatten_df

,target_desc,pid,app
0,PTO油门,DCW_U010314,路谱报表
1,离合器开关,SAE_J000598,路谱报表
2,空档开关,DCW_U010202,"驾驶行为分析, 怠速油耗异常, 怠速过长提醒, 高能耗异常识别模型"
3,空档开关,MD1_U014500,"驾驶行为分析, 怠速油耗异常, 怠速过长提醒, 高能耗异常识别模型"
4,空调开关,DCW_U010203,怠速油耗异常
...,...,...,...
186,SCR子状态,V51_U160403,喷嘴堵塞预警提醒
187,SCR子状态,MD1_U124C00,喷嘴堵塞预警提醒
188,GOC入口温度,SAE_J004753,路谱报表
189,GOC出口温度,SAE_J004754,路谱报表


In [6]:
# 对etl_map_all数据，分割pid
# TODO: 发现e系列 中冷后温度，pid数量与单位数量不匹配
# 读取etl_map_all
etl_map_df = pd.read_sql(
    sql=f"SELECT * FROM dmp.etl_map_all",
    con=mysql_engine_prod
)
# print(etl_map_df.head())

etl_map_flatten_list = []
for _, row in etl_map_df.iterrows():
    if row['signal_code'] is None:
        continue

    for pid_id, pid in enumerate(row['signal_code'].split(';')):
        # print(f"pid_id = {pid_id}")
        if not row['signal_unit'] is None:
#             print(row)
            units = row['signal_unit'].split(';')
            etl_map_flatten_list.append([row['target_desc'], row['target_name'], row['target_unit'], pid,
                                         row['hos_series'], units[pid_id]])
        else:
            etl_map_flatten_list.append([row['target_desc'], row['target_name'], row['target_unit'], pid,
                                         row['hos_series'], None])

etl_map_flatten_df = pd.DataFrame(etl_map_flatten_list, columns=['target_desc', 'target_name', 'target_unit', 'pid',
                                                                 'hos_series', 'signal_unit'])
etl_map_flatten_df

,target_desc,target_name,target_unit,pid,hos_series,signal_unit
0,gps经度,lng,°,longitude,E,°
1,gps纬度,lat,°,latitude,E,°
2,大气压力,atmospheric_pressure,kpa,SAE_J000108,E,kpa
3,大气温度,atmospheric_temperature,℃,SAE_J000171,E,℃
4,车速,speed,km/h,SAE_J000084,E,km/h
...,...,...,...,...,...,...
358,t6(SCR入口温度),t6_temperature,℃,SAE_A003241,all,℃
359,节气门开度,throttle_position,%,SAE_J000051,all,%
360,累计里程,total_distance,km,SAE_J000245,all,km
361,累计油耗,total_fuel,L,SAE_J000250,all,L


In [7]:
# combine 'app pid' with 'etl map', get target_name
merged_df = pd.merge(
    left=app_pid_flatten_df,
    right=etl_map_flatten_df,
    on='pid',
    how='left'
)
merged_df = pd.merge(
    left=merged_df,
    right=para_df,
    left_on='pid',
    right_on='PID',
    how='left'
)
merged_df.loc[~merged_df['target_name'].isna()]

,target_desc_x,pid,app,target_desc_y,target_name,target_unit,hos_series,signal_unit,PID,Description,ECUName,name,unitname
9,PTO开关,DCW_U01020D,"运行参数（天）, 单车油耗报表, 驾驶行为分析, 经济性模型, DPF再生优化, 怠速油耗异...",pto工作状态,pto,None,E,None,DCW_U01020D,PTO开关,DC1W,XPTOSW,-
10,PTO开关,DCW_U01020D,"运行参数（天）, 单车油耗报表, 驾驶行为分析, 经济性模型, DPF再生优化, 怠速油耗异...",pto工作状态,pto,None,D,None,DCW_U01020D,PTO开关,DC1W,XPTOSW,-
11,PTO开关,DCW_U01020D,"运行参数（天）, 单车油耗报表, 驾驶行为分析, 经济性模型, DPF再生优化, 怠速油耗异...",pto工作状态,pto,None,H,None,DCW_U01020D,PTO开关,DC1W,XPTOSW,-
12,PTO开关,DCW_U01020D,"运行参数（天）, 单车油耗报表, 驾驶行为分析, 经济性模型, DPF再生优化, 怠速油耗异...",pto工作状态,pto,None,None,None,DCW_U01020D,PTO开关,DC1W,XPTOSW,-
13,PTO开关,DCW_U01020D,"运行参数（天）, 单车油耗报表, 驾驶行为分析, 经济性模型, DPF再生优化, 怠速油耗异...",pto工作状态,pto,None,all,None,DCW_U01020D,PTO开关,DC1W,XPTOSW,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,油箱液位,SAE_S00096,"运行参数（天）, 加油行为识别",油箱液位,fuel_level,%,D,%,SAE_S00096,油箱液位,J1939,-,%
432,油箱液位,SAE_S00096,"运行参数（天）, 加油行为识别",油箱液位,fuel_level,%,H,%,SAE_S00096,油箱液位,J1939,-,%
433,油箱液位,SAE_S00096,"运行参数（天）, 加油行为识别",油箱液位,fuel_level,%,R,%,SAE_S00096,油箱液位,J1939,-,%
434,油箱液位,SAE_S00096,"运行参数（天）, 加油行为识别",油箱液位,fuel_level,%,None,%,SAE_S00096,油箱液位,J1939,-,%


In [8]:
# 对同含义的pid进行整合
merged_gdf = merged_df.groupby('target_desc_x')
desc_pid_dict = {}
for desc, group in merged_gdf:
    desc_pid_dict[desc] = {'target_unit': group['unitname'].values[0], 'signal_unit': {}}
    desc_pid_dict[desc]['app'] = group['app'].values[0]
    for _, row in group.iterrows():
        unit = row['signal_unit']
        if isinstance(row['signal_unit'], float):
            unit = None
        
        if row['signal_unit'] is None:
            unit = None

        desc_pid_dict[desc]['signal_unit'][row['pid']] = unit
# print(desc_pid_dict)

desc_pid_list = []
for desc in desc_pid_dict.keys():
    signal_unit_dict = desc_pid_dict[desc]['signal_unit']
    for pid in signal_unit_dict.keys():
        etl_match_df = etl_map_flatten_df.loc[etl_map_flatten_df['pid']==pid]
        if etl_match_df.shape[0] > 0:
            target_name = etl_match_df['target_name'].values[0]
        else:
            target_name = None
        desc_pid_list.append([desc, target_name, desc_pid_dict[desc]['target_unit'], pid, desc_pid_dict[desc]['signal_unit'][pid], desc_pid_dict[desc]['app']])
desc_pid_df = pd.DataFrame(desc_pid_list, columns=['desc', 'target_name', 'target_unit', 'pid', 'unit', 'app'])
desc_pid_df.to_csv("/data1/pythonProjects/pyetl/data/desc_pid.csv", index=False, encoding='utf-8-sig')
desc_pid_df

,desc,target_name,target_unit,pid,unit,app
0,DCU故障码,None,-,V51_U190201,None,None
1,DOC上游排气温度,t4_temperature,℃,SAE_J004765,℃,"运行参数（天）, DPF分析, DPF再生分析, DPF再生超温场景识别, DOC效率监控,..."
2,DOC上游排气温度,t4_temperature,℃,MD1_U112400,℃,"运行参数（天）, DPF分析, DPF再生分析, DPF再生超温场景识别, DOC效率监控,..."
3,DPF上游排气温度,t5_temperature,℃,SAE_J004766,℃,"运行参数（天）, DPF再生优化, DPF分析, DPF再生分析, DPF再生超温场景识别,..."
4,DPF上游排气温度,t5_temperature,℃,MD1_U112C00,℃,"运行参数（天）, DPF再生优化, DPF分析, DPF再生分析, DPF再生超温场景识别,..."
...,...,...,...,...,...,...
185,驻车再生请求标志,Parking_regeneration,-,DCW_U010706,None,"DPF再生优化, DPF分析, DPF再生分析, DOC升温异常, DPF再生分析-适配常州东风"
186,驻车再生请求计数器,None,-,DCW_U010726,None,"DPF分析, DPF再生分析"
187,驾驶员报警系统严重等级,None,-,SAE_J005826,None,None
188,驾驶员报警系统指示状态,None,-,SAE_J005825,None,None


# 查看23年 SDECData2M_all中出现过的PID

In [9]:
def date_range(start: str, end: str):
    """
    获取两个日期之间的所有日期

    :param start: 开始日期
    :param end: 结束日期
    :return: 日期列表
    """
    import datetime
    from datetime import timedelta
    start_date = datetime.datetime.strptime(start, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end, '%Y-%m-%d')
    delta = end_date - start_date  # as timedelta
    days = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    return days


start_date = '2022-01-01'
end_date = '2023-08-06'

date_list = date_range(start_date, end_date)

from sqlalchemy import text

pid_dict = {}
for dt in date_list:
    sql = f"""
        select pid,count(pid) as ct from (
        SELECT `params.codeM` as pid from sdecdmp.SDECbasicdatabycodeM_all sma
        where uploadDate = '{dt.strftime('%Y-%m-%d')}' )
        where pid like 'SA%'
        OR pid like 'WW%'
        OR pid like 'DCW%'
        OR pid like 'MD1%'
        OR pid like 'C81%'
        OR pid like 'V51%'
        OR pid like 'F17%'
        group by pid
        having ct > 100
        """
    date_pid_df = pd.read_sql(
        sql=text(sql), con=ck_engine
    )
    for p in date_pid_df['pid']:
        pid_dict[p] = 1
print(len(pid_dict.keys()))

346


In [10]:
# 与EOL系统中para进行合并
sdecdmp_pid_df = pd.DataFrame(columns=['pid'])
sdecdmp_pid_df.loc[:, 'pid'] = list(pid_dict.keys())

sdecdmp_pid_desc_df = pd.merge(left=sdecdmp_pid_df, right=para_df.loc[:, ['PID', 'Description', 'unitname']], left_on='pid', right_on='PID', how='left')
sdecdmp_pid_desc_df = sdecdmp_pid_desc_df.loc[:, ['pid', 'Description', 'unitname']]
print(sdecdmp_pid_desc_df)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
new_pid_df = pd.merge(left=sdecdmp_pid_desc_df, right=desc_pid_df, on='pid', how='left')

old_pid_df = pd.read_excel("/data1/pythonProjects/pyetl/data/PID中文名整理.xlsx", sheet_name='Sheet1')
old_pid_df = old_pid_df.loc[:, ['pid', 'target_name']]
new_pid_df = new_pid_df.merge(old_pid_df, on='pid', how='left')

new_pid_df = new_pid_df.loc[:, ['pid', 'Description', 'desc', 'target_name_y', 'target_unit', 'unitname', 'app']]
new_pid_df.to_csv("/data1/pythonProjects/pyetl/data/new_desc_pid.csv", index=False, encoding='utf-8-sig')
new_pid_df

             pid   Description unitname
0    DCW_U01020B         巡航开关2        -
1    SAE_J006595         尿素喷射量    g/min
2    SAE_J516150      尿素喷嘴喷射状态        -
3    SAE_J000174          燃油温度     degC
4    SAE_J516127       发动机运行模式        -
..           ...           ...      ...
341   WW_U015410        RPM最高值      rpm
342   WW_U015409  PostOCEGT最高值     degC
343   WW_U015405     DRVP电压最高值        V
344   WW_U014C14          空调开关        -
345  SAE_J000523            挡位        -

[346 rows x 3 columns]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,pid,Description,desc,target_name_y,target_unit,unitname,app
0,DCW_U01020B,巡航开关2,NaN,NaN,NaN,-,NaN
1,SAE_J006595,尿素喷射量,尿素喷射量,catalyst_flow_rate,g/min,g/min,"运行参数（天）, SCR分析, SCR效率分析, 尿素倒计时, DOC升温异常, 尿素品质异..."
2,SAE_J516150,尿素喷嘴喷射状态,尿素喷嘴喷射状态,catalyst_injection_status,-,-,SCR分析
3,SAE_J000174,燃油温度,燃油温度,fuel_temperature,degC,degC,"经济性模型, 智能精滤更换提醒, 机油更换识别-海平面"
4,SAE_J516127,发动机运行模式,发动机运行模式,engine_state,-,-,"SCR分析, 驾驶行为分析, 经济性模型, DPF再生优化, DPF分析, DPF再生分析,..."
...,...,...,...,...,...,...,...
344,WW_U015410,RPM最高值,NaN,NaN,NaN,rpm,NaN
345,WW_U015409,PostOCEGT最高值,NaN,NaN,NaN,degC,NaN
346,WW_U015405,DRVP电压最高值,NaN,NaN,NaN,V,NaN
347,WW_U014C14,空调开关,NaN,NaN,NaN,-,NaN
